In [1]:
### Import the required libraries
import numpy as np
import scipy
import matplotlib.pyplot as plt
import cmocean

import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter("ignore")

import tensorflow as tf
import tensorflow.keras as keras
from keras import metrics
import innvestigate

import os
from os.path import join
import sys

import xarray as xr
import xmitgcm
from xmitgcm import open_mdsdataset
import ecco_v4_py as ecco

import random

# See if GPUs are available
from keras import backend as K
if bool(K._get_available_gpus()):
    print("Running on GPU")
else:
    print("Running on CPU")

# Append to sys.path the absolute path to src/XAIRT
path_list = os.path.abspath('').split('/')
path_src_XAIRT = ''
for link in path_list[:-1]:
    path_src_XAIRT = path_src_XAIRT+link+'/'
sys.path.append(path_src_XAIRT+'/src')

# Now import module XAIRT
from XAIRT import *

### https://stackoverflow.com/questions/36288235/how-to-get-stable-results-with-tensorflow-setting-random-seed ###
### https://keras.io/examples/keras_recipes/reproducibility_recipes/ ###
SEED = 42
keras.utils.set_random_seed(SEED)
tf.config.experimental.enable_op_determinism()

tf.compat.v1.disable_eager_execution()

2024-01-29 11:05:51.118413: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-29 11:05:52.635163: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1532] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 38221 MB memory:  -> device: 0, name: NVIDIA A100-PCIE-40GB, pci bus id: 0000:21:00.0, compute capability: 8.0
2024-01-29 11:05:52.637822: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1532] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 38221 MB memory:  -> device: 1, name: NVIDIA A100-PCIE-40GB, pci bus id: 0000:81:00.0, compute capability: 8.0
2024-01-29 11:05:52.639915: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1532] Created device /job:localhost/rep

Running on GPU


## Linear Regression using ANN

In [2]:
N = 10000
X = np.random.rand(N,2)
y = np.dot(X,np.array([1,2]))[:,np.newaxis]

In [3]:
Layers = [{'size': X.shape[1], 'activation': None     , 'use_bias': None},
          {'size': 8         , 'activation': 'relu'   , 'use_bias': True, 'l2_w_reg': 10  , 'l2_b_reg': 10},
          {'size': 8         , 'activation': 'relu'   , 'use_bias': True, 'l2_w_reg': 0.01, 'l2_b_reg': 0.01},
          {'size': 1         , 'activation': 'linear', 'use_bias': True, 'l2_w_reg': 0.01, 'l2_b_reg': 0.01}]
Losses = [{'kind': 'mse', 'weight': 1.0}]

NNkwargs = {'losses': Losses, 'metrics': ['mae'],
            'batch_size': 128, 'epochs': 100, 'validation_split': 0.2,
            'filename': 'model_simpleTests_regr', 'dirname': os.path.abspath(''),
            'random_nn_seed': 42}

keras.backend.clear_session()
sgd = keras.optimizers.SGD(learning_rate=0.01, momentum=0.9, nesterov=True)
adam = keras.optimizers.Adam()
NNkwargs['optimizer'] = sgd

K = TrainFullyConnectedNN(X, y, layers = Layers, **NNkwargs)
L = TrainLR(X, y, y_ref = 0.0, fit_intercept = False)
best_model = K.quickTrain()
regr = L.quickTrain()
y_NN = best_model.predict(X)
y_LR = regr.predict(X)

2024-01-29 11:05:53.249670: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1532] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 38221 MB memory:  -> device: 0, name: NVIDIA A100-PCIE-40GB, pci bus id: 0000:21:00.0, compute capability: 8.0
2024-01-29 11:05:53.251673: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1532] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 38221 MB memory:  -> device: 1, name: NVIDIA A100-PCIE-40GB, pci bus id: 0000:81:00.0, compute capability: 8.0
2024-01-29 11:05:53.253406: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1532] Created device /job:localhost/replica:0/task:0/device:GPU:2 with 38221 MB memory:  -> device: 2, name: NVIDIA A100-PCIE-40GB, pci bus id: 0000:e2:00.0, compute capability: 8.0
2024-01-29 11:05:53.266786: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:354] MLIR V1 optimization pass is not enabled



Epoch 1: val_loss improved from inf to 0.51722, saving model to /home1/07665/shrey911/XAIRT/examples/model_simpleTests_regr.h5


2024-01-29 11:05:54.134772: I tensorflow/stream_executor/cuda/cuda_blas.cc:1786] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.



Epoch 2: val_loss improved from 0.51722 to 0.49832, saving model to /home1/07665/shrey911/XAIRT/examples/model_simpleTests_regr.h5

Epoch 3: val_loss improved from 0.49832 to 0.48959, saving model to /home1/07665/shrey911/XAIRT/examples/model_simpleTests_regr.h5

Epoch 4: val_loss improved from 0.48959 to 0.48904, saving model to /home1/07665/shrey911/XAIRT/examples/model_simpleTests_regr.h5

Epoch 5: val_loss improved from 0.48904 to 0.48670, saving model to /home1/07665/shrey911/XAIRT/examples/model_simpleTests_regr.h5

Epoch 6: val_loss improved from 0.48670 to 0.46904, saving model to /home1/07665/shrey911/XAIRT/examples/model_simpleTests_regr.h5

Epoch 7: val_loss improved from 0.46904 to 0.46551, saving model to /home1/07665/shrey911/XAIRT/examples/model_simpleTests_regr.h5

Epoch 8: val_loss improved from 0.46551 to 0.46410, saving model to /home1/07665/shrey911/XAIRT/examples/model_simpleTests_regr.h5

Epoch 9: val_loss did not improve from 0.46410

Epoch 10: val_loss did not 

In [4]:
normalizeDict = {'bool_': True, 'kind': 'Sum'}
kwargs = {'y_ref': 0.00}
lrp_methods = [dict(name='lrp.z', optParams = {}, title = 'LRP-Z'),
               dict(name='lrp.alpha_1_beta_0', optParams = {}, title = 'LRP-A1B0')]

for method in lrp_methods:

    Xplain = XAIR(best_model, method, 'classic', X, normalizeDict, **kwargs)
    a, *_  = Xplain.quick_analyze()
    print(f"{method['name']}: {np.mean(a, axis = 0)}")

Xplain = XLR(regr, X)
a, *_  = Xplain.quick_analyze()
print(f"lrp.LR: {np.mean(a, axis = 0)}")

lrp.z: [0.32214266 0.67785734]
lrp.alpha_1_beta_0: [0.33478287 0.66521713]
lrp.LR: [0.36919998 0.63080002]


## Optimal input for Classification problem

In [5]:
N = 1000000
X = np.random.rand(N,2)
y = np.dot(X,np.array([1,2]))[:,np.newaxis]
oneHot = np.zeros([y.shape[0],2])
oneHot[:,0] = ((y[:,0] >= 1.) & (y[:,0] <= 2.))
oneHot[:,1] = ((y[:,0] < 1.) | (y[:,0] > 2.))

np.sum(oneHot[:,0]), np.sum(oneHot[:,1])

(500060.0, 499940.0)

In [6]:
Layers = [{'size': X.shape[1], 'activation': None     , 'use_bias': None},
          {'size': 8         , 'activation': 'relu'   , 'use_bias': True},
          {'size': 8         , 'activation': 'relu'   , 'use_bias': True},
          {'size': 2         , 'activation': 'softmax', 'use_bias': True}]
Losses = [{'kind': 'categorical_crossentropy', 'weight': 1.0}]

NNkwargs = {'losses': Losses, 'metrics': ['mae'],
            'batch_size': 2048, 'epochs': 100, 'validation_split': 0.2,
            'filename': 'model_simpleTests_OI', 'dirname': os.path.abspath(''),
            'random_nn_seed': 42}

# learning rate schedule
def step_decay(epoch):
    initial_lrate = 0.01
    drop = 0.5
    epochs_drop = 25
    lrate = initial_lrate * drop**np.floor((1+epoch)/epochs_drop)
    return lrate

keras.backend.clear_session()
sgd = keras.optimizers.SGD(learning_rate=0.01, momentum=0.9, nesterov=True)
NNkwargs['optimizer'] = sgd

K = TrainFullyConnectedNN(X, oneHot, layers = Layers, **NNkwargs)
best_model = K.quickTrain(step_decay)
oneHot_NN = best_model.predict(X)

2024-01-29 11:06:24.666393: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1532] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 38221 MB memory:  -> device: 0, name: NVIDIA A100-PCIE-40GB, pci bus id: 0000:21:00.0, compute capability: 8.0
2024-01-29 11:06:24.670561: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1532] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 38221 MB memory:  -> device: 1, name: NVIDIA A100-PCIE-40GB, pci bus id: 0000:81:00.0, compute capability: 8.0
2024-01-29 11:06:24.672517: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1532] Created device /job:localhost/replica:0/task:0/device:GPU:2 with 38221 MB memory:  -> device: 2, name: NVIDIA A100-PCIE-40GB, pci bus id: 0000:e2:00.0, compute capability: 8.0



Epoch 1: val_loss improved from inf to 0.69943, saving model to /home1/07665/shrey911/XAIRT/examples/model_simpleTests_OI.h5

Epoch 2: val_loss improved from 0.69943 to 0.56074, saving model to /home1/07665/shrey911/XAIRT/examples/model_simpleTests_OI.h5

Epoch 3: val_loss improved from 0.56074 to 0.51931, saving model to /home1/07665/shrey911/XAIRT/examples/model_simpleTests_OI.h5

Epoch 4: val_loss improved from 0.51931 to 0.51361, saving model to /home1/07665/shrey911/XAIRT/examples/model_simpleTests_OI.h5

Epoch 5: val_loss improved from 0.51361 to 0.51125, saving model to /home1/07665/shrey911/XAIRT/examples/model_simpleTests_OI.h5

Epoch 6: val_loss improved from 0.51125 to 0.51016, saving model to /home1/07665/shrey911/XAIRT/examples/model_simpleTests_OI.h5

Epoch 7: val_loss improved from 0.51016 to 0.50975, saving model to /home1/07665/shrey911/XAIRT/examples/model_simpleTests_OI.h5

Epoch 8: val_loss improved from 0.50975 to 0.50944, saving model to /home1/07665/shrey911/XAI

Class 1: $x_1 + 2x_2 \in [1,2]$, Class 2 is the opposite.

In [7]:
inp_numpy = np.array([[0.,0.]])
desired_labels_numpy = np.array([[1.,0.]])
desired_labels = tf.convert_to_tensor(desired_labels_numpy)

print(f"Desired label : {desired_labels_numpy}")
for i in range(100):
    inp = tf.convert_to_tensor(inp_numpy)
    grads = GradientDescent_useGradientTape(best_model, inp, desired_labels, 
                            keras.losses.BinaryCrossentropy())
    grads_numpy = np.squeeze(tf_to_numpy(grads))
    inp_numpy[0,:] = inp_numpy[0,:] - 0.01*grads_numpy
    if (i+1)%10 == 0:
        print(f"Iter {i+1}, Prediction {tf_to_numpy(best_model.predict(inp_numpy))}")
    
print(f"Optimal input is : {inp_numpy}")

inp_numpy = np.array([[0.3,0.6]]) # Optimal input for opposite class to start off
desired_labels_numpy = np.array([[0.,1.]])
desired_labels = tf.convert_to_tensor(desired_labels_numpy)

print(f"Desired label : {desired_labels_numpy}")
for i in range(100):
    inp = tf.convert_to_tensor(inp_numpy)
    grads = GradientDescent_useGradientTape(best_model, inp, desired_labels, 
                            keras.losses.BinaryCrossentropy())
    grads_numpy = np.squeeze(tf_to_numpy(grads))
    inp_numpy[0,:] = inp_numpy[0,:] - 0.01*grads_numpy
    if (i+1)%10 == 0:
        print(f"Iter {i+1}, Prediction {tf_to_numpy(best_model.predict(inp_numpy))}")
    
print(f"Optimal input is : {inp_numpy}")

Desired label : [[1. 0.]]
Iter 10, Prediction [[0.03070879 0.96929127]]
Iter 20, Prediction [[0.88991606 0.11008391]]
Iter 30, Prediction [[0.9233681  0.07663193]]
Iter 40, Prediction [[0.92370385 0.07629617]]
Iter 50, Prediction [[0.9240383  0.07596175]]
Iter 60, Prediction [[0.92445374 0.07554622]]
Iter 70, Prediction [[0.9247855  0.07521456]]
Iter 80, Prediction [[0.92519754 0.07480247]]
Iter 90, Prediction [[0.92552596 0.0744741 ]]
Iter 100, Prediction [[0.92593426 0.07406577]]
Optimal input is : [[0.30620484 0.60838789]]
Desired label : [[0. 1.]]
Iter 10, Prediction [[0.9241213  0.07587866]]
Iter 20, Prediction [[0.92277753 0.07722246]]
Iter 30, Prediction [[0.15050589 0.8494941 ]]
Iter 40, Prediction [[0.03070879 0.96929127]]
Iter 50, Prediction [[0.02648051 0.97351944]]
Iter 60, Prediction [[0.02285772 0.9771423 ]]
Iter 70, Prediction [[0.0199405  0.98005956]]
Iter 80, Prediction [[0.01764037 0.98235965]]
Iter 90, Prediction [[0.0155831 0.9844169]]
Iter 100, Prediction [[0.01383